In [6]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

f = open('TrafficDataset/coordinates.txt', 'r')
lines=f.readlines()

NUM_TEST = round(0.15*len(lines)) # 85% train, 15% test

x_train = np.zeros((len(lines)-NUM_TEST, 32, 32, 3))
y_train = np.zeros((len(lines)-NUM_TEST, 4))
x_test = np.zeros((NUM_TEST, 32, 32, 3))
y_test = np.zeros((NUM_TEST, 4))

for i in range(len(lines)-NUM_TEST):
    line = lines[i]
    coords = [int(j) for j in line.split(';')[1:]]
    img_path = 'TrafficDataset/'+str(i).zfill(6)+'.ppm'
    img = cv2.imread(img_path)
    x_train[i] = img
    y_train[i,:] = coords

for i in range(NUM_TEST):
    line = lines[len(lines)-NUM_TEST+i]
    coords = [int(j) for j in line.split(';')[1:]]
    img_path = 'TrafficDataset/'+str(len(lines)-NUM_TEST+i).zfill(6)+'.ppm'
    img = cv2.imread(img_path)
    x_test[i] = img
    y_test[i,:] = coords

# Number of training examples
n_train = len(x_train) 

# Number of testing examples.
n_test = len(x_test) 

# What's the shape of the image
image_shape = x_train[0].shape 

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)

def normalize(img,img2):
    img_array = np.asarray(img)
    img_array2 = np.asarray(img2)
    normalized = np.empty(np.shape(img_array))
    normalized2 = np.empty(np.shape(img_array2))
    for i in range(3):
        normalized = (img_array - img_array[:,:,:,i].mean()) / (img_array[:,:,:,i].std())
        normalized2 = (img_array2 - img_array[:,:,:,i].mean()) / (img_array[:,:,:,i].std())
    return normalized,normalized2

x_train,x_test=normalize(x_train,x_test)

3639
Number of training examples = 3093
Number of testing examples = 546
Image data shape = (32, 32, 3)


In [7]:
class LossHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.lr = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(step_decay(len(self.losses)))
        print('lr:', step_decay(len(self.losses)))

def step_decay(epoch):
    initial_lrate = 0.05
    if epoch<20:
        lrate = initial_lrate
    elif epoch<40:
        lrate = initial_lrate/5
    elif epoch<45:
        lrate = initial_lrate/25
    elif epoch<50:
        lrate = initial_lrate/(25*5)
    else:
        lrate = initial_lrate/(25*25)
    return lrate

loss_history = LossHistory()
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
callbacks_list = [loss_history, lrate]

model=tf.keras.models.Sequential()

my_init_1=tf.keras.initializers.RandomNormal(mean=0.0, stddev=tf.math.sqrt(2.0/(5*5*192)))
my_init_2=tf.keras.initializers.RandomNormal(mean=0.0, stddev=tf.math.sqrt(2.0/160))
my_init_3=tf.keras.initializers.RandomNormal(mean=0.0, stddev=tf.math.sqrt(2.0/96))
my_init_4=tf.keras.initializers.RandomNormal(mean=0.0, stddev=tf.math.sqrt(2.0/192))
my_init_5=tf.keras.initializers.RandomNormal(mean=0.0, stddev=tf.math.sqrt(2.0/(3*3*192)))
my_init_dense=tf.keras.initializers.VarianceScaling(scale=2.0, mode='fan_out', distribution='uniform')

model.add(tf.keras.layers.Conv2D(192,kernel_size=5,strides=[1,1],kernel_initializer=my_init_1,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False , input_shape=(32,32,3), padding='same',data_format='channels_last',trainable = False))
model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05,trainable = False))
model.add(tf.keras.layers.ReLU(trainable = False))
model.add(tf.keras.layers.Conv2D(160,kernel_size=1,strides=[1,1], kernel_initializer=my_init_2, kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same',trainable = False))
model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05,trainable = False))
model.add(tf.keras.layers.ReLU(trainable = False))
model.add(tf.keras.layers.Conv2D(96,kernel_size=1,strides=[1,1], kernel_initializer=my_init_3,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same',trainable = False))
model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05,trainable = False))
model.add(tf.keras.layers.ReLU(trainable = False))
model.add(tf.keras.layers.ZeroPadding2D(padding=(1, 1),trainable = False))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2,2),trainable = False))

model.add(tf.keras.layers.Conv2D(192,kernel_size=5,strides=[1,1], kernel_initializer=my_init_1,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same',trainable = False))
model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05,trainable = False))
model.add(tf.keras.layers.ReLU(trainable = False))
model.add(tf.keras.layers.Conv2D(192,kernel_size=1,strides=[1,1], kernel_initializer=my_init_4,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same',trainable = False))
model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05,trainable = False))
model.add(tf.keras.layers.ReLU(trainable = False))
model.add(tf.keras.layers.Conv2D(192,kernel_size=1,strides=[1,1], kernel_initializer=my_init_4,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same',trainable = False))
model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05,trainable = False))
model.add(tf.keras.layers.ReLU(trainable = False))
model.add(tf.keras.layers.ZeroPadding2D(padding=(1, 1),trainable = False))
model.add(tf.keras.layers.AveragePooling2D(pool_size=(3, 3), strides=(2,2),trainable = False))

model.add(tf.keras.layers.Conv2D(192,kernel_size=3,strides=[1,1], kernel_initializer=my_init_5,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same', name='conv1'))
model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05, name='bn1'))
model.add(tf.keras.layers.ReLU(name='relu1'))
model.add(tf.keras.layers.Conv2D(192,kernel_size=1,strides=[1,1], kernel_initializer=my_init_4,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same', name='conv2'))
model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05, name='bn2'))
model.add(tf.keras.layers.ReLU(name='relu2'))
model.add(tf.keras.layers.Conv2D(192,kernel_size=1,strides=[1,1], kernel_initializer=my_init_4,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same', name='conv3'))
model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05, name='bn3'))
model.add(tf.keras.layers.ReLU(name='relu3'))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(200, activation=None,kernel_initializer=my_init_dense, kernel_regularizer=tf.keras.regularizers.l2(5e-4),use_bias=False))
model.add(tf.keras.layers.BatchNormalization(beta_initializer='zeros', gamma_initializer='ones', name='bn4'))
model.add(tf.keras.layers.ReLU(name='relu4'))
model.add(tf.keras.layers.Dense(200, activation=None,kernel_initializer=my_init_dense, kernel_regularizer=tf.keras.regularizers.l2(5e-4),use_bias=False))
model.add(tf.keras.layers.BatchNormalization(beta_initializer='zeros', gamma_initializer='ones', name='bn5'))
model.add(tf.keras.layers.ReLU(name='relu5'))
model.add(tf.keras.layers.Dense(4, activation=None,kernel_initializer=my_init_dense, kernel_regularizer=tf.keras.regularizers.l2(5e-4), bias_initializer='zeros',bias_regularizer=tf.keras.regularizers.l2(5e-4)))

my_optimizer=tf.keras.optimizers.SGD(momentum=0.9, nesterov=True)
model.compile(optimizer=my_optimizer,loss='mean_squared_error',metrics=['accuracy'])
model.fit(x_train,y_train, batch_size=16, epochs=100, callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
3093/3093 [==============================] - 6s 2ms/sample - loss: 108.4476 - acc: 0.2244
Epoch 2/100
3093/3093 [==============================] - 4s 1ms/sample - loss: 98.2036 - acc: 0.2292
Epoch 3/100
3093/3093 [==============================] - 4s 1ms/sample - loss: 89.4787 - acc: 0.2431
Epoch 4/100
3093/3093 [==============================] - 4s 1ms/sample - loss: 82.2202 - acc: 0.2444
Epoch 5/100
3093/3093 [==============================] - 4s 1ms/sample - loss: 76.0477 - acc: 0.2299
Epoch 6/100
3093/3093 [==============================] - 4s 1ms/sample - loss: 71.0847 - acc: 0.2176
Epoch 7/100
3093/3093 [==============================] - 4s 1ms/sample - loss: 67.0521 - acc: 0.2299
Epoch 8/100
3093/3093 [==============================] - 4s 1ms/sample - loss: 63.6429 - acc: 0.2325
Epoch 9/100
3093/3093 [==============================] - 4s 1ms/sample - loss: 60.8398 - acc: 0

In [8]:
val_loss, val_acc=model.evaluate(x_test,y_test)
print(val_loss, val_acc)

546/546 [==============================] - 1s 1ms/sample - loss: 44.7982 - acc: 0.3370
44.79820043263418 0.33699635


In [9]:
# 123.18091941218806 0.22511849